In [1]:
!pip install pandasql
!pip install pandas
!pip install plotly
!pip install optimparallel
!pip install torch
!pip install tensorflow
!pip install tensorflow_probability


In [2]:
import pandas as pd
import numpy as np
import functools
import pickle

from BB_solver import bb_ll

from bs4 import BeautifulSoup
from aia import AIASession
from urllib.request import urlopen
from scipy.optimize import Bounds
from optimparallel import minimize_parallel

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier, StackingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import xgboost as XGBoost

In [3]:
ncaa_2011_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_11.csv')
ncaa_2012_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_12.csv')
ncaa_2013_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_13.csv')
ncaa_2014_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_14.csv')
ncaa_2015_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_15.csv')
ncaa_2016_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_16.csv')
ncaa_2017_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_17.csv')
ncaa_2018_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_18.csv')
ncaa_2019_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_19.csv')
ncaa_2021_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_21.csv')
ncaa_2022_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_22.csv')

In [169]:
filter = [
    # 'Team', 'Rating', 'Rating_opp',
    'Win',
    'EFG_alphas_team1', 'EFG_betas_team1', 'EFG_avgs_team1',
    'EFG2_alphas_team1', 'EFG2_betas_team1', 'EFG2_avgs_team1',
    'EFG3_alphas_team1', 'EFG3_betas_team1', 'EFG3_avgs_team1',
    'EORB_alphas_team1', 'EORB_betas_team1', 'EORB_avgs_team1',
    'EDRB_alphas_team1', 'EDRB_betas_team1', 'EDRB_avgs_team1',
    'EAST_alphas_team1', 'EAST_betas_team1', 'EAST_avgs_team1',
    'ETOV_alphas_team1', 'ETOV_betas_team1', 'ETOV_avgs_team1',
    'EFG_alphas_team2', 'EFG_betas_team2', 'EFG_avgs_team2',
    'EFG2_alphas_team2', 'EFG2_betas_team2', 'EFG2_avgs_team2',
    'EFG3_alphas_team2', 'EFG3_betas_team2', 'EFG3_avgs_team2',
    'EORB_alphas_team2', 'EORB_betas_team2', 'EORB_avgs_team2',
    'EDRB_alphas_team2', 'EDRB_betas_team2', 'EDRB_avgs_team2',
    'EAST_alphas_team2', 'EAST_betas_team2', 'EAST_avgs_team2',
    'ETOV_alphas_team2', 'ETOV_betas_team2', 'ETOV_avgs_team2',
    ]

filter_predict = [
    # 'Team',
    # 'Rating', 'Rating_opp',
    'EFG_avgs_team1',
    'EFG2_avgs_team1',
    'EFG3_avgs_team1',
    # 'EFT_avgs_team1',
    'EORB_avgs_team1',
    'EDRB_avgs_team1',
    'EAST_avgs_team1',
    'ETOV_avgs_team1',
    'EBLK_avgs_team1',

    'EFG_avgs_team2',
    'EFG2_avgs_team2',
    'EFG3_avgs_team2',
    # 'EFT_avgs_team2',
    'EORB_avgs_team2',
    'EDRB_avgs_team2',
    'EAST_avgs_team2',
    'ETOV_avgs_team2',
    'EBLK_avgs_team2'
]

filter_train = [
    # 'Team',
    # 'Rating', 'Rating_opp',
    'EFG%_team1',
    'EFG2%_team1',
    'EFG3%_team1',
    # 'EFT%_team1',
    'EORB%_team1',
    'EDRB%_team1',
    'EAST%_team1',
    'ETOV%_team1',
    'EBLK%_team1',

    'EFG%_team2',
    'EFG2%_team2',
    'EFG3%_team2',
    # 'EFT%_team2',
    'EORB%_team2',
    'EDRB%_team2',
    'EAST%_team2',
    'ETOV%_team2',
    'EBLK%_team2',
]

In [170]:
frames = [ncaa_2011_df, ncaa_2012_df, ncaa_2013_df, ncaa_2014_df,
          ncaa_2015_df, ncaa_2016_df, ncaa_2017_df, ncaa_2018_df,
          ncaa_2019_df, ncaa_2021_df, ncaa_2022_df]
combined_df = pd.concat(frames)
# combined_df = combined_df[filter]
combined_df.shape

(114822, 105)

In [171]:
import pandas as pd
import numpy as np
import functools
import pickle

from BB_solver import bb_ll

from bs4 import BeautifulSoup
from aia import AIASession
from urllib.request import urlopen
from scipy.optimize import Bounds
from optimparallel import minimize_parallel

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier, StackingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import xgboost as XGBoost

In [172]:
ncaa_2011_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_11.csv')
ncaa_2012_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_12.csv')
ncaa_2013_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_13.csv')
ncaa_2014_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_14.csv')
ncaa_2015_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_15.csv')
ncaa_2016_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_16.csv')
ncaa_2017_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_17.csv')
ncaa_2018_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_18.csv')
ncaa_2019_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_19.csv')
ncaa_2021_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_21.csv')
ncaa_2022_df = pd.read_csv('/Users/kylesullivan/Documents/Professional/Projects/BB_Models/Data/NCAA_df_22.csv')

In [173]:
ncaa_tour_2011_df = ncaa_2011_df[ncaa_2011_df['DayNum'] > 135]
ncaa_tour_2012_df = ncaa_2012_df[ncaa_2012_df['DayNum'] > 135]
ncaa_tour_2013_df = ncaa_2013_df[ncaa_2013_df['DayNum'] > 135]
ncaa_tour_2014_df = ncaa_2014_df[ncaa_2014_df['DayNum'] > 135]
ncaa_tour_2015_df = ncaa_2015_df[ncaa_2015_df['DayNum'] > 135]
ncaa_tour_2016_df = ncaa_2016_df[ncaa_2016_df['DayNum'] > 135]
ncaa_tour_2017_df = ncaa_2017_df[ncaa_2017_df['DayNum'] > 135]
ncaa_tour_2018_df = ncaa_2018_df[ncaa_2018_df['DayNum'] > 135]
ncaa_tour_2019_df = ncaa_2019_df[ncaa_2019_df['DayNum'] > 135]
ncaa_tour_2021_df = ncaa_2021_df[ncaa_2021_df['DayNum'] > 136]
ncaa_tour_2022_df = ncaa_2022_df[ncaa_2022_df['DayNum'] > 135]

In [174]:
ncaa_2011_df = ncaa_2011_df[ncaa_2011_df['DayNum'] <= 135]
ncaa_2012_df = ncaa_2012_df[ncaa_2012_df['DayNum'] <= 135]
ncaa_2013_df = ncaa_2013_df[ncaa_2013_df['DayNum'] <= 135]
ncaa_2014_df = ncaa_2014_df[ncaa_2014_df['DayNum'] <= 135]
ncaa_2015_df = ncaa_2015_df[ncaa_2015_df['DayNum'] <= 135]
ncaa_2016_df = ncaa_2016_df[ncaa_2016_df['DayNum'] <= 135]
ncaa_2017_df = ncaa_2017_df[ncaa_2017_df['DayNum'] <= 135]
ncaa_2018_df = ncaa_2018_df[ncaa_2018_df['DayNum'] <= 135]
ncaa_2019_df = ncaa_2019_df[ncaa_2019_df['DayNum'] <= 135]
ncaa_2021_df = ncaa_2021_df[ncaa_2021_df['DayNum'] <= 136]
ncaa_2022_df = ncaa_2022_df[ncaa_2022_df['DayNum'] <= 135]

In [175]:
frames = [ncaa_2011_df, ncaa_2012_df, ncaa_2013_df, ncaa_2014_df,
          ncaa_2015_df, ncaa_2016_df, ncaa_2017_df, ncaa_2018_df,
          ncaa_2019_df, ncaa_2021_df, ncaa_2022_df]
combined_df = pd.concat(frames)

y_train = combined_df['Win']

predict_scaler = StandardScaler()
predict_scaler.fit(combined_df[filter_predict])


combined_df['EFG%_team1'] = combined_df['EFG_N'] / combined_df['EFG_D']
combined_df['EFG2%_team1'] = combined_df['EFG2_N'] / combined_df['EFG2_D']
combined_df['EFG3%_team1'] = combined_df['EFG3_N'] / combined_df['EFG3_D']
combined_df['EFT%_team1'] = combined_df['EFT_N'] / combined_df['EFT_D']
combined_df['EORB%_team1'] = combined_df['EORB_N'] / combined_df['EORB_D']
combined_df['EDRB%_team1'] = combined_df['EDRB_N'] / combined_df['EDRB_D']
combined_df['EAST%_team1'] = combined_df['EAST_N'] / combined_df['EAST_D']
combined_df['ETOV%_team1'] = combined_df['ETOV_N'] / combined_df['ETOV_D']
combined_df['EBLK%_team1'] = combined_df['EBLK_N'] / combined_df['EBLK_D']

combined_df['EFG%_team2'] = combined_df['EFG_N_Opp'] / combined_df['EFG_D_Opp']
combined_df['EFG2%_team2'] = combined_df['EFG2_N_Opp'] / combined_df['EFG2_D_Opp']
combined_df['EFG3%_team2'] = combined_df['EFG3_N_Opp'] / combined_df['EFG3_D_Opp']
combined_df['EFT%_team2'] = combined_df['EFT_N_Opp'] / combined_df['EFT_D_Opp']
combined_df['EORB%_team2'] = combined_df['EORB_N_Opp'] / combined_df['EORB_D_Opp']
combined_df['EDRB%_team2'] = combined_df['EDRB_N_Opp'] / combined_df['EDRB_D_Opp']
combined_df['EAST%_team2'] = combined_df['EAST_N_Opp'] / combined_df['EAST_D_Opp']
combined_df['ETOV%_team2'] = combined_df['ETOV_N_Opp'] / combined_df['ETOV_D_Opp']
combined_df['EBLK%_team2'] = combined_df['EBLK_N_Opp'] / combined_df['EBLK_D_Opp']


train_scaler = StandardScaler()
scaled_X_train = train_scaler.fit_transform(combined_df[filter_train])


frames = [ncaa_tour_2011_df, ncaa_tour_2012_df, ncaa_tour_2013_df, ncaa_tour_2014_df,
          ncaa_tour_2015_df, ncaa_tour_2016_df, ncaa_tour_2017_df, ncaa_tour_2018_df,
          ncaa_tour_2019_df, ncaa_tour_2021_df, ncaa_tour_2022_df]
combined_tour_df = pd.concat(frames)


scaled_X_test = predict_scaler.transform(combined_tour_df[filter_predict])
y_test = combined_tour_df['Win']

scaled_X_test_11 = predict_scaler.transform(ncaa_tour_2011_df[filter_predict])
scaled_X_test_12 = predict_scaler.transform(ncaa_tour_2012_df[filter_predict])
scaled_X_test_13 = predict_scaler.transform(ncaa_tour_2013_df[filter_predict])
scaled_X_test_14 = predict_scaler.transform(ncaa_tour_2014_df[filter_predict])
scaled_X_test_15 = predict_scaler.transform(ncaa_tour_2015_df[filter_predict])
scaled_X_test_16 = predict_scaler.transform(ncaa_tour_2016_df[filter_predict])
scaled_X_test_17 = predict_scaler.transform(ncaa_tour_2017_df[filter_predict])
scaled_X_test_18 = predict_scaler.transform(ncaa_tour_2018_df[filter_predict])
scaled_X_test_19 = predict_scaler.transform(ncaa_tour_2019_df[filter_predict])
scaled_X_test_21 = predict_scaler.transform(ncaa_tour_2021_df[filter_predict])
scaled_X_test_22 = predict_scaler.transform(ncaa_tour_2022_df[filter_predict])

y_test_11 = ncaa_tour_2011_df['Win']
y_test_12 = ncaa_tour_2012_df['Win']
y_test_13 = ncaa_tour_2013_df['Win']
y_test_14 = ncaa_tour_2014_df['Win']
y_test_15 = ncaa_tour_2015_df['Win']
y_test_16 = ncaa_tour_2016_df['Win']
y_test_17 = ncaa_tour_2017_df['Win']
y_test_18 = ncaa_tour_2018_df['Win']
y_test_19 = ncaa_tour_2019_df['Win']
y_test_21 = ncaa_tour_2021_df['Win']
y_test_22 = ncaa_tour_2022_df['Win']

# combined_tour_df = combined_tour_df[filter_predict]
# combined_tour_df = combined_tour_df.rename(columns={'EFG_avgs_team1':'EFG%_team1', 'EFG_avgs_team2':'EFG%_team2',
#                                                     'EFG2_avgs_team1':'EFG2%_team1', 'EFG2_avgs_team2':'EFG2%_team2',
#                                                     'EFG3_avgs_team1':'EFG3%_team1', 'EFG3_avgs_team2':'EFG3%_team2',
#                                                     'EFT_avgs_team1':'EFT%_team1', 'EFT_avgs_team2':'EFT%_team2',
#                                                     'EORB_avgs_team1':'EORB%_team1', 'EORB_avgs_team2':'EORB%_team2',
#                                                     'EDRB_avgs_team1':'EDRB%_team1', 'EDRB_avgs_team2':'EDRB%_team2',
#                                                     'EAST_avgs_team1':'EAST%_team1', 'EAST_avgs_team2':'EAST%_team2',
#                                                     'ETOV_avgs_team1':'ETOV%_team1', 'ETOV_avgs_team2':'ETOV%_team2',
#                                                     'EBLK_avgs_team1':'EBLK%_team1', 'EBLK_avgs_team2':'EBLK%_team2'})

In [176]:
combined_df

,GameId,Team,DayNum,NumOT,Score,Home,Conf,OppConf,Seed,EFG_N,...,EBLK%_team1,EFG%_team2,EFG2%_team2,EFG3%_team2,EFT%_team2,EORB%_team2,EDRB%_team2,EAST%_team2,ETOV%_team2,EBLK%_team2
0,1,Illinois,7,0.0,79,1,big_ten,big_west,9.0,33,...,0.126984,0.412698,0.348837,0.350000,0.222222,0.268293,0.742857,0.545455,0.170732,0.017241
1,1,UC Irvine,7,0.0,65,-1,big_west,big_ten,NaN,26,...,0.017241,0.568966,0.617647,0.333333,0.224138,0.257143,0.731707,0.586207,0.160920,0.126984
2,2,Maryland,7,0.0,105,1,big_ten,wac,NaN,36,...,0.116883,0.389610,0.409836,0.187500,0.220779,0.352941,0.484848,0.392857,0.266055,0.016129
3,2,Seattle,7,0.0,76,-1,wac,big_ten,NaN,30,...,0.016129,0.580645,0.611111,0.250000,0.532258,0.515152,0.647059,0.657143,0.236842,0.116883
4,3,Texas,7,0.0,83,1,big_twelve,patriot,4.0,32,...,0.079365,0.333333,0.375000,0.173913,0.158730,0.170213,0.666667,0.421053,0.168539,0.084746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10693,5347,TAM C. Christi,134,0.0,67,0,southland,swac,16.0,29,...,0.054545,0.454545,0.452381,0.307692,0.472727,0.216216,0.829268,0.391304,0.096386,0.134328
10694,5348,Wright St,135,0.0,93,0,horizon,aec,16.0,34,...,0.056338,0.464789,0.574468,0.166667,0.225352,0.256410,0.647059,0.225806,0.166667,0.081967
10695,5348,Bryant,135,0.0,82,0,aec,horizon,16.0,33,...,0.081967,0.557377,0.500000,0.428571,0.426230,0.352941,0.743590,0.655172,0.138298,0.056338
10696,5349,Rutgers,135,2.0,87,0,big_ten,acc,11.0,42,...,0.083333,0.555556,0.584906,0.315789,0.125000,0.200000,0.628571,0.459459,0.149425,0.028169


In [177]:
def ParameterSearch(regressor, degree, scaler_bool, interaction_only, X_train, y_train):
    """
    This is a k=5 cross fold validation function to find optimal parameters for
    different regressors.

    Parameters:
    ------------
    regressor: (string)
      type or regressor to be run ("rfr", "etr", "gbr", "xgb", "knnr", "ablr",
                                   "svr", "lr")
    scaler_bool: bool
      True to apply standard scaler to the input data first
    degree: int
      degree to which the X_train input will be converted prior to running the
      regressor
    interaction_only: bool
      if the X_train input degree will only add interaction terms
    X_train: DataFrame
      training inputs
    y_train: DataFrame
      training outputs

    Returns:
    ------------
    None
    """

    # all of the regressor options to be run
    rfc = RandomForestClassifier(random_state=42, n_jobs = -1)
    etc = ExtraTreesClassifier(random_state=42, n_jobs = -1)
    nnc = MLPClassifier(random_state=42,  max_iter=10000)
    gbc = GradientBoostingClassifier(random_state=42)
    xgb = XGBoost.XGBClassifier(random_state=42, n_jobs=-1)
    knnc = KNeighborsClassifier()
    absvc = AdaBoostClassifier(random_state=42, base_estimator=SVC())
    svc = SVC()
    lr = LogisticRegression(random_state=0, warm_start=True, max_iter=10000)

    # parameter values to be tried
    rfc_grid = {
        'rfc__max_depth': [4, 6, 8, 10],
        'rfc__max_features': ['sqrt', None],
        'rfc__min_samples_leaf': [4, 8, 16, 32],
        'rfc__n_estimators': [50, 100, 200],
        'rfc__bootstrap': [True],
        'rfc__max_samples': [0.25, 0.5, 1]
    }
    etc_grid = {
        'etc__max_depth': [4, 6, 8, 10],
        'etc__max_features': ['sqrt', None],
        'etc__min_samples_leaf': [4, 8, 16, 32],
        'etc__n_estimators': [50, 100, 200],
        'etc__bootstrap': [True],
        'etc__max_samples': [0.25, 0.5, 1]
    }
    nnc_grid = {
        'nnc__hidden_layer_sizes': [(i, j) for i in range(5, 51, 5) for j in range(2, 21, 2)],
        'nnc__alpha': [i/2 for i in list(range(20))],
    }
    gbc_grid = {
        'gbc__max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
        'gbc__max_features': ['sqrt', None],
        'gbc__min_samples_leaf': [1, 2, 4, 8, 16],
        'gbc__n_estimators': [50, 100, 200, 300],
        'gbc__subsample': [0.25, 0.5, 0.75, 1]
    }
    xgb_grid = {
        'xgb__max_depth': [5, 10, 20],
        'xgb__n_estimators': [100, 200, 300],
        'xgb__gamma': [1],
        'xgb__eta': [0.05, 0.1],
        'xgb__subsample': [0.25],
        'xgb__min_child_weight': [0.25, 0.5],
        'xgb__reg_lambda': [0.5, 1],
        'xgb__alpha': [0.5, 1]
    }
    knnc_grid = {
        'knnc__n_neighbors': list(range(1, 51, 1))
    }
    absvc_grid = {
        'absvc__n_estimators': list(range(10, 101, 10)),
        'absvc__learning_rate': [0.05, 0.1, 0.2]
    }
    svc_grid = {
        'svc__degree': list(range(1, 11, 1))
    }
    lr_grid = {
        'lr__C': [0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25, 3.5, 3.75, 4, 4.25, 4.5, 4.75, 5]
    }

    # check if scaler will be used
    if scaler_bool == True:
        scaler = StandardScaler()
    else:
        scaler = None

    if degree:
        poly = PolynomialFeatures(degree=degree, interaction_only=interaction_only)
    else:
        poly = None

    # set up the grid and pipe for the chosen regressor
    if regressor=='rfc':
        grid = rfc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('rfc', rfc)])
    elif regressor=='etc':
        grid = etc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('etc', etc)])
    elif regressor=='nnc':
        grid = nnc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('nnc', nnc)])
    elif regressor=='gbc':
        grid = gbc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('gbc', gbc)])
    elif regressor=='xgb':
        grid = xgb_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('xgb', xgb)])
    elif regressor=='knnc':
        grid = knnc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('knnc', knnc)])
    elif regressor=='absvc':
        grid = absvc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('absvc', absvc)])
    elif regressor=='svc':
        grid = svc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('svc', svc)])
    elif regressor=='lr':
        grid = lr_grid
        pipe = Pipeline(steps=[('poly', poly), ('scaler', scaler), ('lr', lr)])

    # create the GridSearchCV object, and fit the passed input to it
    gs = GridSearchCV(pipe, grid, n_jobs=-1, cv=3, verbose=1)
    gs.fit(X_train, y_train)

    # print out the best score on the cross validation holdout data and the best parameters
    print("Best score:", gs.best_score_)
    print()
    print(gs.best_params_)

In [178]:
# X_train = combined_df.drop(columns='Win')
# y_train = combined_df['Win']
# X_test = combined_tour_df.drop(columns='Win')
# y_test = combined_tour_df['Win']

In [179]:
ParameterSearch('lr', 2, False, False, scaled_X_train, y_train)

Fitting 3 folds for each of 26 candidates, totalling 78 fits


/Users/kylesullivan/anaconda3/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best score: 0.9186218668896204

{'lr__C': 3}


In [180]:
def model(X_train, y_train, X_test, y_test, version):

    # y_train = X_train['Win'].to_list()
    # y_test = X_test['Win'].to_list()

    if version == 0:
        poly = PolynomialFeatures(2)
        X_train = poly.fit_transform(X_train)
        X_test = poly.transform(X_test)

        # scaler = StandardScaler()
        # X_train = scaler.fit_transform(X_train)
        # X_test = scaler.transform(X_test)
        clf = LogisticRegression(random_state=0, warm_start=True, C=3, max_iter=10000).fit(X_train, y_train)

    if version == 1:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        clf = MLPClassifier(hidden_layer_sizes=(7, 3), alpha=0.05, random_state=0, max_iter=10000).fit(X_train, y_train)

    if version == 2:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        clf = RandomForestClassifier(bootstrap=True, max_depth=8, max_features='sqrt', max_samples=0.75, min_samples_leaf=8, n_estimators=500, random_state=0).fit(X_train, y_train)

    # clf = ExtraTreesClassifier(bootstrap=True, max_depth=10, max_features='sqrt', max_samples=0.5, min_samples_leaf=16, n_estimators=300, random_state=0).fit(X_train, y_train)

    # clf = GradientBoostingClassifier(random_state=0).fit(X_train, y_train)

    # pred = clf.predict(X)
    train_acc = clf.score(X_train, y_train)
    test_acc = clf.score(X_test, y_test)
    train_pred = clf.predict(X_train)
    test_pred = clf.predict(X_test)
    y_train_arr = np.array(y_train)
    y_test_arr = np.array(y_test)

    # output = [(pred[i], y_test[i]) for i in range(len(y_test))]
    # print(output)
    # print("Train Acc:", train_acc)
    # print("Test Acc:", test_acc)
    # print('Train Total Incorrect:', (len(train_pred)-sum(train_pred==y_train_arr))/2)
    print('Test Total Correct:', sum(test_pred==y_test_arr)/2)

    # return clf

In [181]:
clf = model(scaled_X_train, y_train, scaled_X_test, y_test, 0)

Test Total Correct: 492.0


In [298]:
for pair in [(scaled_X_test_11, y_test_11, 2011), (scaled_X_test_12, y_test_12, 2012), (scaled_X_test_13, y_test_13, 2013),
             (scaled_X_test_14, y_test_14, 2014), (scaled_X_test_15, y_test_15, 2015), (scaled_X_test_16, y_test_16, 2016),
             (scaled_X_test_17, y_test_17, 2017), (scaled_X_test_18, y_test_18, 2018), (scaled_X_test_19, y_test_19, 2019),
             (scaled_X_test_21, y_test_21, 2021), (scaled_X_test_22, y_test_22, 2022)]:
    print("Year:", pair[2])
    model(scaled_X_train, y_train, pair[0], pair[1], 0)
    print("--------------------------------------------------",)

Year: 2011
Test Total Correct: 42.0
--------------------------------------------------
Year: 2012
Test Total Correct: 46.0
--------------------------------------------------
Year: 2013
Test Total Correct: 45.0
--------------------------------------------------
Year: 2014
Test Total Correct: 43.0
--------------------------------------------------
Year: 2015
Test Total Correct: 50.0
--------------------------------------------------
Year: 2016
Test Total Correct: 46.0
--------------------------------------------------
Year: 2017
Test Total Correct: 45.0
--------------------------------------------------
Year: 2018
Test Total Correct: 44.0
--------------------------------------------------
Year: 2019
Test Total Correct: 48.0
--------------------------------------------------
Year: 2021
Test Total Correct: 41.0
--------------------------------------------------
Year: 2022
Test Total Correct: 42.0
--------------------------------------------------


In [ ]:
bl_df = ncaa_2019_df
bl_df = ncaa_2019_df[ncaa_2019_df['Home'] == 0]
bl_df = bl_df.drop_duplicates(subset=['Team'])
bl_df = bl_df[filter]
scaler = StandardScaler()
teams = bl_df['Team'].to_list()
bl_df = bl_df.drop(columns=['Team'])
cols = bl_df.columns
bl_df = scaler.fit_transform(bl_df)
bl_df = pd.DataFrame(data=bl_df, columns=cols)
bl_df['Team'] = teams
scores_weight = 0
bl_df['Combined_Rating'] = (scores_weight)*bl_df['Rating'] + (1-scores_weight)*((177/465)*(bl_df['EFG_alphas_team1'] + bl_df['EFG_betas_team1']) + (73/465)*(bl_df['EORB_alphas_team1'] + bl_df['EORB_betas_team1']) + (44/465)*(bl_df['EDRB_alphas_team1'] + bl_df['EDRB_betas_team1']) + (141/465)*(bl_df['ETOV_alphas_team1'] + bl_df['ETOV_betas_team1']) + (30/465)*(bl_df['EAST_alphas_team1'] + bl_df['EAST_betas_team1']))
bl_df = bl_df.sort_values(by='Combined_Rating', ascending=False)
bl_df[[
    'Team', 'Combined_Rating', 'Rating',
    'EFG_alphas_team1', 'EFG_betas_team1', 'EFG_avgs_team1',
    'EFG2_alphas_team1', 'EFG2_betas_team1', 'EFG2_avgs_team1',
    'EFG3_alphas_team1', 'EFG3_betas_team1', 'EFG3_avgs_team1',
    'EORB_alphas_team1', 'EORB_betas_team1', 'EORB_avgs_team1',
    'EDRB_alphas_team1', 'EDRB_betas_team1', 'EDRB_avgs_team1',
    'EAST_alphas_team1', 'EAST_betas_team1', 'EAST_avgs_team1',
    'ETOV_alphas_team1', 'ETOV_betas_team1', 'ETOV_avgs_team1',
    'EFG_alphas_team2', 'EFG_betas_team2', 'EFG_avgs_team2',
    'EFG2_alphas_team2', 'EFG2_betas_team2', 'EFG2_avgs_team2',
    'EFG3_alphas_team2', 'EFG3_betas_team2', 'EFG3_avgs_team2',
    'EORB_alphas_team2', 'EORB_betas_team2', 'EORB_avgs_team2',
    'EDRB_alphas_team2', 'EDRB_betas_team2', 'EDRB_avgs_team2',
    'EAST_alphas_team2', 'EAST_betas_team2', 'EAST_avgs_team2',
    'ETOV_alphas_team2', 'ETOV_betas_team2', 'ETOV_avgs_team2',
]]

In [ ]:
ncaa_2019_df = ncaa_2019_df.reset_index()
ncaa_2019_df = ncaa_2019_df.drop(columns=['index'])
ncaa_2019_df = ncaa_2019_df.reset_index()
ncaa_2019_df = ncaa_2019_df[ncaa_2019_df['index'] % 2 == 0]
ncaa_2019_df = ncaa_2019_df.drop(columns=['index'])

In [ ]:
scaler = StandardScaler()

train_2019 = ncaa_2019_df[ncaa_2019_df['DayNum'] <= 135]
test_2019 = ncaa_2019_df[ncaa_2019_df['DayNum'] > 135]

y_train_2019 = train_2019['Win'].to_list()
y_test_2019 = test_2019['Win'].to_list()

X_train_2019 = train_2019[filter]
X_train_2019 = X_train_2019.drop(columns=['Team'])

temp = train_2019.drop_duplicates(subset=['Team'])
teams_2019 = temp['Team'].to_list()
temp = temp[filter]
temp = temp.drop(columns=['Team'])
temp

temp.iloc[:, 22:] = 1.5

for metric in ['EFG', 'EFG2', 'EFG3', 'EORB', 'EDRB', 'EAST', 'ETOV']:
    temp[metric+'_avgs_team1'] = temp.apply(lambda row : row[metric+'_alphas_team1'] / (row[metric+'_alphas_team1'] + row[metric+'_betas_team2']), axis=1)
    temp[metric+'_avgs_team2'] = temp.apply(lambda row : row[metric+'_alphas_team2'] / (row[metric+'_alphas_team2'] + row[metric+'_betas_team1']), axis=1)

In [ ]:
temp

In [ ]:
def EnsemblePredictor(X_train, y_train, X_test, y_test=None):
    """
    An ensemble regressor estimation function.

    Parameters:
    ------------
    X_train: DataFrame
      training inputs
    y_train: DataFrame
      training outputs
    X_test: DataFrame
      testing inputs
    y_test: DataFrame
      testing outputs

    Returns:
    ------------
    None (if y_test != None)

    """

    # convert the input into a polynomial of degree 2 with only interation terms (meaning x1, x2, and x1*x2)
    # poly = PolynomialFeatures(degree=2, interaction_only=True)
    # X_train_poly = poly.fit_transform(X_train)
    # X_test_poly = poly.transform(X_test)

    # normalize the data
    scaler = StandardScaler()
    # X_train = scaler.fit_transform(X_train)
    # X_test = scaler.transform(X_test)
    # X_train_poly = scaler.fit_transform(X_train)
    # X_test_poly = scaler.transform(X_test)
    # X_train = scaler.fit_transform(X_train_poly)
    # X_test = scaler.transform(X_test_poly)

    poly_class=Pipeline([('poly_class', PolynomialFeatures(degree=2, interaction_only=False)), ('scaler', scaler), ('lr', LogisticRegression(random_state=10, warm_start=True, C=0.01, max_iter=20000),)])

    # MLP_class=Pipeline([('scaler', scaler), ('MLP_class', MLPClassifier(hidden_layer_sizes=(20, 5), alpha=1, random_state=0, max_iter=100000),)])
    #
    # RF_class=Pipeline([('scaler', scaler), ('RF_class', RandomForestClassifier(bootstrap=True, max_depth=8, max_features='sqrt', max_samples=0.5, min_samples_leaf=8, n_estimators=300, random_state=0),)])

    # poly_class = LogisticRegression(random_state=0, warm_start=True, C=0.01, max_iter=10000)
    # MLP_class = MLPClassifier(hidden_layer_sizes=(20, 5), alpha=.5, random_state=0, max_iter=100000)
    # RF_class = RandomForestClassifier(bootstrap=True, max_depth=8, max_features='sqrt', max_samples=0.75, min_samples_leaf=4, n_estimators=300, random_state=0)

    # estimators to be used in the ensemble
    estimators = [
        # ('1', MLP_class,),]
        # ('1', RF_class,),]
        ('1', poly_class,),]

    # create regressors stack
    stack_reg = StackingClassifier(
        estimators=estimators
    )

    # fit the training data to the stack
    stack_reg.fit(X_train, y_train)

    # pred = clf.predict(X)
    train_acc = stack_reg.score(X_train, y_train)

    train_pred = stack_reg.predict(X_train)
    test_pred = stack_reg.predict(X_test)
    test_pred_prob = stack_reg.predict_proba(X_test)
    y_train_arr = np.array(y_train)

    # output = [(pred[i], y_test[i]) for i in range(len(y_test))]
    # print(output)
    print("Train Acc:", train_acc)
    if y_test:
        test_acc = stack_reg.score(X_test, y_test)
        print("Test Acc:", test_acc)
    print('Train Total Incorrect:', len(train_pred)-sum(train_pred==y_train_arr))
    if y_test:
        y_test_arr = np.array(y_test)
        print('Test Total Correct:', sum(test_pred==y_test_arr))
    else:
        return test_pred_prob[:,1]

In [ ]:
rankings = EnsemblePredictor(X_train_2019, y_train_2019, temp)
d = {'Team': teams_2019, 'Ranking': rankings}
rankings_2019_df = pd.DataFrame(data=d)
rankings_2019_df = rankings_2019_df.sort_values(by='Ranking', ascending=False)
rankings_2019_df

In [ ]:
rankings_2019_df[0:50]

In [ ]:
bl_df[0:50]

In [47]:
combined_df

,GameId,Team,DayNum,NumOT,Score,Home,Conf,OppConf,Seed,EFG_N,...,ETOV_avgs_team1,ETOV_avgs_team2,EBLK_alphas_team1,EBLK_betas_team1,EBLK_alphas_team2,EBLK_betas_team2,EBLK_avgs_team1,EBLK_avgs_team2,EDRB_avgs_team1,EDRB_avgs_team2
0,1,Illinois,7,0.0,79,1,big_ten,big_west,9.0,33,...,0.161096,0.145609,61.392977,832.580803,16.417355,556.415837,0.099372,0.019337,0.759926,0.660153
1,1,UC Irvine,7,0.0,65,-1,big_west,big_ten,NaN,26,...,0.145609,0.161096,16.417355,556.415837,61.392977,832.580803,0.019337,0.099372,0.660153,0.759926
2,2,Maryland,7,0.0,105,1,big_ten,wac,NaN,36,...,0.227828,0.190504,51.371591,755.894528,18.531403,329.215997,0.134980,0.023929,0.651490,0.632422
3,2,Seattle,7,0.0,76,-1,wac,big_ten,NaN,30,...,0.190504,0.227828,18.531403,329.215997,51.371591,755.894528,0.023929,0.134980,0.632422,0.651490
4,3,Texas,7,0.0,83,1,big_twelve,patriot,4.0,32,...,0.188819,0.172844,57.985201,805.143301,25.462317,499.718606,0.103971,0.030655,0.818079,0.559488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10819,5410,Duke,152,0.0,77,0,acc,acc,2.0,32,...,0.122954,0.110741,86.822446,1055.747312,59.138629,1045.640328,0.076667,0.053045,0.708204,0.788976
10820,5411,Villanova,152,0.0,65,0,big_east,big_twelve,2.0,28,...,0.138876,0.131763,44.122615,1191.880893,70.066077,1202.427499,0.035396,0.055522,0.711131,0.725754
10821,5411,Kansas,152,0.0,81,0,big_twelve,big_east,1.0,36,...,0.131763,0.138876,70.066077,1202.427499,44.122615,1191.880893,0.055522,0.035396,0.725754,0.711131
10822,5412,North Carolina,154,0.0,69,0,acc,big_twelve,8.0,26,...,0.110235,0.139025,59.138629,1045.640328,70.066077,1202.427499,0.046877,0.062800,0.771956,0.738580


In [44]:
[0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]

[0.0,
 0.01,
 0.02,
 0.03,
 0.04,
 0.05,
 0.06,
 0.07,
 0.08,
 0.09,
 0.1,
 0.11,
 0.12,
 0.13,
 0.14,
 0.15,
 0.16,
 0.17,
 0.18,
 0.19,
 0.2,
 0.21,
 0.22,
 0.23,
 0.24,
 0.25,
 0.26,
 0.27,
 0.28,
 0.29,
 0.3,
 0.31,
 0.32,
 0.33,
 0.34,
 0.35,
 0.36,
 0.37,
 0.38,
 0.39,
 0.4,
 0.41,
 0.42,
 0.43,
 0.44,
 0.45,
 0.46,
 0.47,
 0.48,
 0.49,
 0.5,
 0.51,
 0.52,
 0.53,
 0.54,
 0.55,
 0.56,
 0.57,
 0.58,
 0.59,
 0.6,
 0.61,
 0.62,
 0.63,
 0.64,
 0.65,
 0.66,
 0.67,
 0.68,
 0.69,
 0.7,
 0.71,
 0.72,
 0.73,
 0.74,
 0.75,
 0.76,
 0.77,
 0.78,
 0.79,
 0.8,
 0.81,
 0.82,
 0.83,
 0.84,
 0.85,
 0.86,
 0.87,
 0.88,
 0.89,
 0.9,
 0.91,
 0.92,
 0.93,
 0.94,
 0.95,
 0.96,
 0.97,
 0.98,
 0.99]

In [45]:
def ParameterSearch(regressor, degree, scaler_bool, interaction_only, X_train, y_train):
    """
    This is a k=5 cross fold validation function to find optimal parameters for
    different regressors.

    Parameters:
    ------------
    regressor: (string)
      type or regressor to be run ("rfr", "etr", "gbr", "xgb", "knnr", "ablr",
                                   "svr", "lr")
    scaler_bool: bool
      True to apply standard scaler to the input data first
    degree: int
      degree to which the X_train input will be converted prior to running the
      regressor
    interaction_only: bool
      if the X_train input degree will only add interaction terms
    X_train: DataFrame
      training inputs
    y_train: DataFrame
      training outputs

    Returns:
    ------------
    None
    """

    # all of the regressor options to be run
    rfc = RandomForestClassifier(random_state=42, n_jobs = -1)
    etc = ExtraTreesClassifier(random_state=42, n_jobs = -1)
    nnc = MLPClassifier(random_state=42,  max_iter=10000)
    gbc = GradientBoostingClassifier(random_state=42)
    xgb = XGBoost.XGBClassifier(random_state=42, n_jobs=-1)
    knnc = KNeighborsClassifier()
    absvc = AdaBoostClassifier(random_state=42, base_estimator=SVC())
    svc = SVC()
    lc = LogisticRegression(random_state=0, warm_start=True, max_iter=10000)

    # parameter values to be tried
    rfc_grid = {
        'rfc__max_depth': [4, 6, 8, 10],
        'rfc__max_features': ['sqrt', None],
        'rfc__min_samples_leaf': [4, 8, 16, 32],
        'rfc__n_estimators': [50, 100, 200],
        'rfc__bootstrap': [True],
        'rfc__max_samples': [0.25, 0.5, 1]
    }
    etc_grid = {
        'etc__max_depth': [4, 6, 8, 10],
        'etc__max_features': ['sqrt', None],
        'etc__min_samples_leaf': [4, 8, 16, 32],
        'etc__n_estimators': [50, 100, 200],
        'etc__bootstrap': [True],
        'etc__max_samples': [0.25, 0.5, 1]
    }
    nnc_grid = {
        'nnc__hidden_layer_sizes': [(i, j) for i in range(5, 51, 5) for j in range(2, 21, 2)],
        'nnc__alpha': [i/2 for i in list(range(20))],
    }
    gbc_grid = {
        'gbc__max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
        'gbc__max_features': ['sqrt', None],
        'gbc__min_samples_leaf': [1, 2, 4, 8, 16],
        'gbc__n_estimators': [50, 100, 200, 300],
        'gbc__subsample': [0.25, 0.5, 0.75, 1]
    }
    xgb_grid = {
        'xgb__max_depth': [5, 10, 20],
        'xgb__n_estimators': [100, 200, 300],
        'xgb__gamma': [1],
        'xgb__eta': [0.05, 0.1],
        'xgb__subsample': [0.25],
        'xgb__min_child_weight': [0.25, 0.5],
        'xgb__reg_lambda': [0.5, 1],
        'xgb__alpha': [0.5, 1]
    }
    knnc_grid = {
        'knnc__n_neighbors': list(range(1, 51, 1))
    }
    absvc_grid = {
        'absvc__n_estimators': list(range(10, 101, 10)),
        'absvc__learning_rate': [0.05, 0.1, 0.2]
    }
    svc_grid = {
        'svc__degree': list(range(1, 11, 1))
    }
    lc_grid = {
        'lr__C': [0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
    }

    # check if scaler will be used
    if scaler_bool == True:
        scaler = StandardScaler()
    else:
        scaler = None

    # set up the grid and pipe for the chosen regressor
    if regressor=='rfc':
        grid = rfc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('rfc', rfc)])
    elif regressor=='etc':
        grid = etc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('etc', etc)])
    elif regressor=='nnc':
        grid = nnc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('nnc', nnc)])
    elif regressor=='gbc':
        grid = gbc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('gbc', gbc)])
    elif regressor=='xgb':
        grid = xgb_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('xgb', xgb)])
    elif regressor=='knnc':
        grid = knnc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('knnc', knnc)])
    elif regressor=='absvc':
        grid = absvc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('absvc', absvc)])
    elif regressor=='svc':
        grid = svc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('svc', svc)])
    elif regressor=='lc':
        grid = lc_grid
        pipe = Pipeline(steps=[('scaler', scaler), ('lc', lc)])

    # create the GridSearchCV object, and fit the passed input to it
    gs = GridSearchCV(pipe, grid, n_jobs=-1, cv=3, verbose=1)
    gs.fit(X_train, y_train)

    # print out the best score on the cross validation holdout data and the best parameters
    print("Best score:", gs.best_score_)
    print()
    print(gs.best_params_)

In [56]:
bl_df = ncaa_2019_df
bl_df = ncaa_2019_df[ncaa_2019_df['Home'] == 0]
bl_df = bl_df.drop_duplicates(subset=['Team'])
bl_df = bl_df[filter]
scaler = StandardScaler()
teams = bl_df['Team'].to_list()
bl_df = bl_df.drop(columns=['Team'])
cols = bl_df.columns
bl_df = scaler.fit_transform(bl_df)
bl_df = pd.DataFrame(data=bl_df, columns=cols)
bl_df['Team'] = teams
scores_weight = 0
bl_df['Combined_Rating'] = (scores_weight)*bl_df['Rating'] + (1-scores_weight)*((177/465)*(bl_df['EFG_alphas_team1'] + bl_df['EFG_betas_team1']) + (73/465)*(bl_df['EORB_alphas_team1'] + bl_df['EORB_betas_team1']) + (44/465)*(bl_df['EDRB_alphas_team1'] + bl_df['EDRB_betas_team1']) + (141/465)*(bl_df['ETOV_alphas_team1'] + bl_df['ETOV_betas_team1']) + (30/465)*(bl_df['EAST_alphas_team1'] + bl_df['EAST_betas_team1']))
bl_df = bl_df.sort_values(by='Combined_Rating', ascending=False)
bl_df[[
    'Team', 'Combined_Rating', 'Rating',
    'EFG_alphas_team1', 'EFG_betas_team1', 'EFG_avgs_team1',
    'EFG2_alphas_team1', 'EFG2_betas_team1', 'EFG2_avgs_team1',
    'EFG3_alphas_team1', 'EFG3_betas_team1', 'EFG3_avgs_team1',
    'EORB_alphas_team1', 'EORB_betas_team1', 'EORB_avgs_team1',
    'EDRB_alphas_team1', 'EDRB_betas_team1', 'EDRB_avgs_team1',
    'EAST_alphas_team1', 'EAST_betas_team1', 'EAST_avgs_team1',
    'ETOV_alphas_team1', 'ETOV_betas_team1', 'ETOV_avgs_team1',
    'EFG_alphas_team2', 'EFG_betas_team2', 'EFG_avgs_team2',
    'EFG2_alphas_team2', 'EFG2_betas_team2', 'EFG2_avgs_team2',
    'EFG3_alphas_team2', 'EFG3_betas_team2', 'EFG3_avgs_team2',
    'EORB_alphas_team2', 'EORB_betas_team2', 'EORB_avgs_team2',
    'EDRB_alphas_team2', 'EDRB_betas_team2', 'EDRB_avgs_team2',
    'EAST_alphas_team2', 'EAST_betas_team2', 'EAST_avgs_team2',
    'ETOV_alphas_team2', 'ETOV_betas_team2', 'ETOV_avgs_team2',
]]

,Team,Combined_Rating,Rating,EFG_alphas_team1,EFG_betas_team1,EFG_avgs_team1,EFG2_alphas_team1,EFG2_betas_team1,EFG2_avgs_team1,EFG3_alphas_team1,...,EORB_avgs_team2,EDRB_alphas_team2,EDRB_betas_team2,EDRB_avgs_team2,EAST_alphas_team2,EAST_betas_team2,EAST_avgs_team2,ETOV_alphas_team2,ETOV_betas_team2,ETOV_avgs_team2
123,Gonzaga,3.501888,2.181244,3.379385,2.523483,2.341211,4.206364,2.553629,3.072060,1.010543,...,0.215665,-0.673344,0.681151,-1.169315,0.390861,0.344971,-1.046181,1.313091,0.228623,0.987373
194,North Carolina,3.220173,2.400679,1.802561,1.400638,0.500459,1.306630,1.486868,-0.291943,1.765091,...,-1.260687,0.363046,0.376420,-1.514376,-0.305427,1.509465,0.300815,0.264504,0.841800,0.081173
2,Michigan St,3.136217,2.208962,2.868737,3.257764,0.676462,2.288020,3.366565,0.085840,2.423005,...,-0.018566,1.411660,0.722945,-0.830982,-0.402314,0.099127,-0.390880,-0.268587,0.363840,-0.280747
165,Virginia,3.117161,2.781147,2.569578,2.662449,1.913709,1.331740,1.728678,1.084759,3.465757,...,-0.023749,-0.153957,1.239851,-0.642475,-0.957361,-0.441882,-1.828995,-0.555739,-1.771221,-0.667126
1,Duke,3.104944,2.594561,1.836329,2.697485,-0.112735,3.543258,2.104853,0.334009,-0.920233,...,1.636355,2.399705,2.502554,-0.701489,0.474928,0.843191,-0.156211,-0.054245,0.351568,-0.228807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,Alcorn St,-2.439889,-1.878770,-1.816592,-1.371413,-0.216206,-1.753725,-1.879769,-0.185640,-1.443521,...,0.033867,-2.319169,-0.676135,-1.526723,0.308977,-2.041487,2.010986,-0.624536,-0.178811,-0.297665
227,Idaho,-2.568934,-1.772001,-0.775299,-0.978746,-0.683129,-1.380303,-1.420387,-0.933769,0.741083,...,1.822784,0.291496,0.917358,1.052135,-0.414106,0.676858,0.946134,-0.794516,0.341540,-0.619538
299,MD E Shore,-2.668773,-2.214909,-2.183772,-1.666683,-0.077945,-1.741245,-1.473071,0.417274,-2.379621,...,0.833908,-1.323724,0.058460,0.974492,-1.025720,-0.055081,0.454361,0.480937,-0.717371,0.655150
83,UNC Asheville,-2.788308,-2.324207,-1.576501,-1.209516,-1.438497,-1.603109,-1.539626,-0.936730,-0.889186,...,2.334282,-0.583475,0.401540,0.006346,-0.174563,-0.520156,2.021733,-0.108590,-0.896815,0.237764


In [49]:
ncaa_2019_df = ncaa_2019_df.reset_index()
ncaa_2019_df = ncaa_2019_df.drop(columns=['index'])
ncaa_2019_df = ncaa_2019_df.reset_index()
ncaa_2019_df = ncaa_2019_df[ncaa_2019_df['index'] % 2 == 0]
ncaa_2019_df = ncaa_2019_df.drop(columns=['index'])

In [50]:
scaler = StandardScaler()

train_2019 = ncaa_2019_df[ncaa_2019_df['DayNum'] <= 135]
test_2019 = ncaa_2019_df[ncaa_2019_df['DayNum'] > 135]

y_train_2019 = train_2019['Win'].to_list()
y_test_2019 = test_2019['Win'].to_list()

X_train_2019 = train_2019[filter]
X_train_2019 = X_train_2019.drop(columns=['Team'])

temp = train_2019.drop_duplicates(subset=['Team'])
teams_2019 = temp['Team'].to_list()
temp = temp[filter]
temp = temp.drop(columns=['Team'])
temp

temp.iloc[:, 22:] = 1.5

for metric in ['EFG', 'EFG2', 'EFG3', 'EORB', 'EDRB', 'EAST', 'ETOV']:
    temp[metric+'_avgs_team1'] = temp.apply(lambda row : row[metric+'_alphas_team1'] / (row[metric+'_alphas_team1'] + row[metric+'_betas_team2']), axis=1)
    temp[metric+'_avgs_team2'] = temp.apply(lambda row : row[metric+'_alphas_team2'] / (row[metric+'_alphas_team2'] + row[metric+'_betas_team1']), axis=1)

In [51]:
temp

,Rating,Rating_opp,EFG_alphas_team1,EFG_betas_team1,EFG_avgs_team1,EFG2_alphas_team1,EFG2_betas_team1,EFG2_avgs_team1,EFG3_alphas_team1,EFG3_betas_team1,...,EORB_avgs_team2,EDRB_alphas_team2,EDRB_betas_team2,EDRB_avgs_team2,EAST_alphas_team2,EAST_betas_team2,EAST_avgs_team2,ETOV_alphas_team2,ETOV_betas_team2,ETOV_avgs_team2
0,1611.491935,1303.760368,118.280273,120.414333,0.975264,370.669698,380.490936,0.991972,211.254464,430.023480,...,0.005739,3.0,3.0,0.013887,3.0,3.0,0.058065,3.0,3.0,0.013430
2,1553.554278,1288.975592,100.435224,107.155229,0.970996,298.254675,327.145328,0.990042,201.028461,414.235965,...,0.006054,3.0,3.0,0.011294,3.0,3.0,0.104529,3.0,3.0,0.013757
4,1660.747090,1362.125396,142.709122,122.055351,0.979411,443.205083,396.937940,0.993277,244.836732,429.153114,...,0.006895,3.0,3.0,0.015247,3.0,3.0,0.070979,3.0,3.0,0.013149
6,1553.158767,1318.398695,115.273174,112.833435,0.974635,356.535115,354.082448,0.991656,214.336664,416.702761,...,0.006499,3.0,3.0,0.020988,3.0,3.0,0.092539,3.0,3.0,0.013463
8,1608.884125,1453.698624,117.519887,122.923750,0.975108,359.848351,404.174569,0.991732,216.134462,415.849186,...,0.006208,3.0,3.0,0.013102,3.0,3.0,0.092181,3.0,3.0,0.013376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4280,1302.870392,1439.098557,90.291664,74.215711,0.967843,265.154663,212.479319,0.988812,187.496261,335.291948,...,0.008467,3.0,3.0,0.015983,3.0,3.0,0.078088,3.0,3.0,0.013583
4446,1362.125396,1396.499356,84.869029,83.820151,0.965858,240.338851,237.518417,0.987672,187.536539,376.157041,...,0.009001,3.0,3.0,0.015195,3.0,3.0,0.090367,3.0,3.0,0.013640
4450,1318.398695,1470.775696,83.929191,84.019904,0.965489,240.540023,220.212549,0.987682,177.494865,404.970528,...,0.008306,3.0,3.0,0.016835,3.0,3.0,0.123818,3.0,3.0,0.014012
5328,1473.737659,1470.775696,95.680168,88.299451,0.969599,287.981029,231.036321,0.989690,189.192208,436.280181,...,0.009383,3.0,3.0,0.017114,3.0,3.0,0.083859,3.0,3.0,0.013134


In [52]:
def EnsemblePredictor(X_train, y_train, X_test, y_test=None):
    """
    An ensemble regressor estimation function.

    Parameters:
    ------------
    X_train: DataFrame
      training inputs
    y_train: DataFrame
      training outputs
    X_test: DataFrame
      testing inputs
    y_test: DataFrame
      testing outputs

    Returns:
    ------------
    None (if y_test != None)

    """

    # convert the input into a polynomial of degree 2 with only interation terms (meaning x1, x2, and x1*x2)
    # poly = PolynomialFeatures(degree=2, interaction_only=True)
    # X_train_poly = poly.fit_transform(X_train)
    # X_test_poly = poly.transform(X_test)

    # normalize the data
    scaler = StandardScaler()
    # X_train = scaler.fit_transform(X_train)
    # X_test = scaler.transform(X_test)
    # X_train_poly = scaler.fit_transform(X_train)
    # X_test_poly = scaler.transform(X_test)
    # X_train = scaler.fit_transform(X_train_poly)
    # X_test = scaler.transform(X_test_poly)

    poly_class=Pipeline([('poly_class', PolynomialFeatures(degree=2, interaction_only=False)), ('scaler', scaler), ('lr', LogisticRegression(random_state=10, warm_start=True, C=0.01, max_iter=20000),)])

    # MLP_class=Pipeline([('scaler', scaler), ('MLP_class', MLPClassifier(hidden_layer_sizes=(20, 5), alpha=1, random_state=0, max_iter=100000),)])
    #
    # RF_class=Pipeline([('scaler', scaler), ('RF_class', RandomForestClassifier(bootstrap=True, max_depth=8, max_features='sqrt', max_samples=0.5, min_samples_leaf=8, n_estimators=300, random_state=0),)])

    # poly_class = LogisticRegression(random_state=0, warm_start=True, C=0.01, max_iter=10000)
    # MLP_class = MLPClassifier(hidden_layer_sizes=(20, 5), alpha=.5, random_state=0, max_iter=100000)
    # RF_class = RandomForestClassifier(bootstrap=True, max_depth=8, max_features='sqrt', max_samples=0.75, min_samples_leaf=4, n_estimators=300, random_state=0)

    # estimators to be used in the ensemble
    estimators = [
        # ('1', MLP_class,),]
        # ('1', RF_class,),]
        ('1', poly_class,),]

    # create regressors stack
    stack_reg = StackingClassifier(
        estimators=estimators
    )

    # fit the training data to the stack
    stack_reg.fit(X_train, y_train)

    # pred = clf.predict(X)
    train_acc = stack_reg.score(X_train, y_train)

    train_pred = stack_reg.predict(X_train)
    test_pred = stack_reg.predict(X_test)
    test_pred_prob = stack_reg.predict_proba(X_test)
    y_train_arr = np.array(y_train)

    # output = [(pred[i], y_test[i]) for i in range(len(y_test))]
    # print(output)
    print("Train Acc:", train_acc)
    if y_test:
        test_acc = stack_reg.score(X_test, y_test)
        print("Test Acc:", test_acc)
    print('Train Total Incorrect:', len(train_pred)-sum(train_pred==y_train_arr))
    if y_test:
        y_test_arr = np.array(y_test)
        print('Test Total Correct:', sum(test_pred==y_test_arr))
    else:
        return test_pred_prob[:,1]

In [36]:
rankings = EnsemblePredictor(X_train_2019, y_train_2019, temp)
d = {'Team': teams_2019, 'Ranking': rankings}
rankings_2019_df = pd.DataFrame(data=d)
rankings_2019_df = rankings_2019_df.sort_values(by='Ranking', ascending=False)
rankings_2019_df

Train Acc: 0.7779403694896653
Train Total Incorrect: 1214


,Team,Ranking
153,Houston,0.931548
51,Texas Tech,0.930398
91,Virginia,0.929420
138,Kansas St,0.928340
45,Georgetown,0.928052
...,...,...
268,Incarnate Word,0.081510
320,Delaware St,0.080506
168,High Point,0.078392
189,Tulane,0.076320


In [38]:
rankings_2019_df[0:50]

,Team,Ranking
153,Houston,0.931548
51,Texas Tech,0.930398
91,Virginia,0.929420
138,Kansas St,0.928340
45,Georgetown,0.928052
25,Wisconsin,0.927952
70,Villanova,0.926931
20,Indiana,0.925656
272,ETSU,0.924874
38,Purdue,0.923831


In [57]:
bl_df[0:50]

,Rating,Rating_opp,EFG_alphas_team1,EFG_betas_team1,EFG_avgs_team1,EFG2_alphas_team1,EFG2_betas_team1,EFG2_avgs_team1,EFG3_alphas_team1,EFG3_betas_team1,...,EDRB_betas_team2,EDRB_avgs_team2,EAST_alphas_team2,EAST_betas_team2,EAST_avgs_team2,ETOV_alphas_team2,ETOV_betas_team2,ETOV_avgs_team2,Team,Combined_Rating
123,2.181244,0.565519,3.379385,2.523483,2.341211,4.206364,2.553629,3.072060,1.010543,1.767191,...,0.681151,-1.169315,0.390861,0.344971,-1.046181,1.313091,0.228623,0.987373,Gonzaga,3.501888
194,2.400679,1.159226,1.802561,1.400638,0.500459,1.306630,1.486868,-0.291943,1.765091,0.843403,...,0.376420,-1.514376,-0.305427,1.509465,0.300815,0.264504,0.841800,0.081173,North Carolina,3.220173
2,2.208962,2.026502,2.868737,3.257764,0.676462,2.288020,3.366565,0.085840,2.423005,1.724844,...,0.722945,-0.830982,-0.402314,0.099127,-0.390880,-0.268587,0.363840,-0.280747,Michigan St,3.136217
165,2.781147,-0.574984,2.569578,2.662449,1.913709,1.331740,1.728678,1.084759,3.465757,3.532600,...,1.239851,-0.642475,-0.957361,-0.441882,-1.828995,-0.555739,-1.771221,-0.667126,Virginia,3.117161
1,2.594561,2.230964,1.836329,2.697485,-0.112735,3.543258,2.104853,0.334009,-0.920233,2.840186,...,2.502554,-0.701489,0.474928,0.843191,-0.156211,-0.054245,0.351568,-0.228807,Duke,3.104944
0,2.215144,2.621400,1.542747,2.010744,-0.750542,1.550578,2.968787,-0.350099,1.020665,-0.010606,...,2.464073,-1.843146,-0.223750,0.740159,-0.749098,0.391670,1.645983,0.350977,Kentucky,2.745745
141,1.939480,0.204653,1.458219,3.166632,0.616740,1.292656,3.216925,0.385147,1.214293,2.084355,...,-0.211929,-0.325281,0.949957,-0.867328,0.078366,-0.377967,1.278212,-0.358115,Texas Tech,2.689864
87,2.235845,-0.983776,1.307952,2.805885,1.453777,1.324681,2.042719,1.106302,0.844135,3.129154,...,-1.302985,0.341490,-0.419962,-0.791693,-1.838594,-0.592379,0.125779,-0.800670,Michigan,2.682268
19,1.803431,-0.727535,1.753924,0.977866,2.082417,1.263164,1.187770,1.704516,1.485325,0.292195,...,-0.809028,-2.176538,-1.076452,1.105106,-0.987823,-0.767575,-0.034253,-0.894104,Purdue,2.604289
163,2.318484,1.529660,2.485710,1.613957,0.332493,2.665898,2.225573,0.641876,1.070246,0.153096,...,0.493183,0.231860,0.132076,1.000385,0.180047,-0.825539,0.594738,-0.989988,Tennessee,2.542995
